### Imports

In [1]:
import snscrape.modules.twitter
import pandas as pd
from gensim.models import Doc2Vec
from gensim.utils import simple_preprocess
import pickle
import csv

### Create function that takes in a twitter handle as input and returns that user's probable political bent

In [2]:
def get_bent(twitter_handle):
    
    # import Doc2Vec model
    
    doc_model = Doc2Vec.load('final_doc2vec')
    
    # import SVC model
    
    svc_filename = "svc_final_model.pkl"
    with open(svc_filename, 'rb') as file:
        svc = pickle.load(file)
    
    # scrape all tweets from user
    
    tweets = snscrape.modules.twitter.TwitterUserScraper(username=twitter_handle)
    
    # create temporary file to hold tweets
    
    with open('temp_file7.csv', 'w', encoding='utf-8') as f:
        wr=csv.writer(f, lineterminator = '\n')
        for i in tweets.get_items():
            wr.writerow([i.content])
                    
    # read temporary file in
    
    df = pd.read_csv('temp_file7.csv', header=None)
    
    # transform rows of tweets into single string 
    
    tweets_list = df[0].tolist()
    one_string = ''.join(tweets_list)
    
    # process tweets with gensim
    
    clean_doc = simple_preprocess(one_string)
    
    # infer document embeddings for entirety of user's tweets
    
    vectors = doc_model.infer_vector(clean_doc)
    
    # make prediction in svc model
    
    prediction = svc.predict([vectors])
    proba = svc.predict_proba([vectors])
    most_similar = doc_model.wv.most_similar([vectors])
    # display political bent

    if prediction == 1:
        print(f'Well it seems to me that this twitter user, @{twitter_handle}, is most likely a Democrat.')
    else:
        print(f'Hey now, looks like Mr. or Ms. @{twitter_handle}, just may be a Republican.')
    
    print(prediction)
    print(one_string[:200])
    print(proba)
    
    

In [3]:
get_bent('CitizenCee')


Well it seems to me that this twitter user, @CitizenCee, is most likely a Democrat.
[1]
@ZackBornstein Why is everything so duuuuuuuuumb@AshleyRoboto https://t.co/pfdhOALwP9@ledansemacabre @svdate Lost the primary and wasn't elected in the first place.@arstechnica @JenLucPiquant Nah. Hon
[[0.0075528 0.9924472]]
